# Advanced Python

In [ ]:
print("Hello, World!")

## Lab 01.01 - Files and Lists

In [ ]:
with open("names.txt", encoding= "utf8") as file:
    names: list[str] = [line.rstrip("\n") for line in file.readlines()]

longest_name = max(names, key = len)
shortest_name = min(names, key = len)

print(f"Longest name is: {longest_name}")
print(f"Shortest name is: {shortest_name}")


In [ ]:
del names
del shortest_name
del longest_name
del file

## Lab 01.02 - Sorting Files

## Lab 01.03 - File Exception Handling
### Part 1 - Random
1. Collect user input on the name of the file to be opened
1. If the file exists, read a random line from the file
1. Otherwise, prompt the nuser to give another file name until they give you a file that exists

In [ ]:
from random import choice

#filename = input("Enter the file to be opened: ")
filename = "students.txt"

file_exists = False
while not file_exists:
    try:
        infile = open(filename, "r")
        file_exists = True
    except FileNotFoundError:
        filename = input("Please enter a file that ACTUALLY exists: ")

lines = [line.rstrip("\n") for line in infile.readlines()]
infile.close()
print(f"Here's a random line from {filename}: {choice(lines)}")

In [ ]:
del choice
del filename
del file_exists
del infile
del lines

# Module 2 - Relational Databases
* Relational databases separate data by table
* CREATE TABLE
* INSERT
* SELECT
* UPDATE
* DELETE

## Connection and Cursor
* SQLite built into Python 3
* Before working with a database, you must connect to it
* A `Cursor` is used to execute the SQL command strings
* The `execute()` function takes a string argument

In [ ]:
import sqlite3
from os import path, remove

DATABASE = "demo.db"
if path.exists(DATABASE):
    remove(DATABASE)

conn = sqlite3.connect(DATABASE)
cursor = conn.cursor()

## Executing Commands
* `CREATE TABLE`

  Possible data types:
  * `INTEGER` - stores integer numbers
  * `TEXT` - stores string values
  * `REAL` - stores floating point numbers

In [ ]:
command = """
CREATE TABLE table_name (column_1 TEXT, column_2 INTEGER)
"""
cursor.execute(command)
print("A new table with two columns has been created")


* `INSERT INTO`
  ```sql
  command = INSERT INTO table_name VALUES ("Value 1", 12345), ("Value 2", 6789)
  ```
  ```python
  cursor.execute(command)
  ```
  * Database will look like this

    | column_1 | column_2 |
    |:---------|:---------|
    | Value 1  | 1234     |
    | Value 2  | 5678     |

  * **Note:** Data will not persist by default it must be committed with `connection.commit()`

In [ ]:
command = """
INSERT INTO table_name VALUES
    ("Value 1", 1234),
    ("Value 2", 5678)
"""
cursor.execute(command)
conn.commit()
print("The entries have been added")


* `SELECT FROM`
  * Selects data from the columns
  * **Note:** `fetchall()` pulls the data from the cursor and returns the rows in a list of tuples

In [ ]:
command = """
SELECT * FROM table_name
"""
cursor.execute(command).fetchall()


* `WHERE`
  * Operators

    | Symbol | Meaning               |
    |:-------|:----------------------|
    | `=`    | equals                |
    | `!=`   | not equals            |
    | `>`    | greater than          |
    | `<`    | less than             |
    | `>=`   | greater than or equal |
    | `<=`   | less than or equal    |

* Dynamic Statements
  * The question mark can be used to indicate a place for dynamic option input in an SQL statement
  

In [ ]:
command = """
SELECT * FROM table_name
WHERE column_2 > ?
"""
cursor.execute(command, (2000,)).fetchall()

In [ ]:
del command
del DATABASE
cursor.close()
conn.close()
del cursor
del conn

## Lab 02.01 - Corrupted Data
1. Bring the rock_store_backup.db, new_equipment.csv, updated_prices.csv into your environment
1. THe company recently started selling mining equipment and wanted to add the new stock to the database
   * Examine the contents of new_equipment.csv
   * Create a table called equipment with areas for all the data in `new_equpment.csv`
1. Next open the file and loop through the data, inserting it into the `equpment` table
1. Next use `updated_prices.csv` to update the `rocks` table (already in the database) entries' price values in the csv
1. Next delete all the entries in the rocks table that have a stock of 0. They will not be restocked.

_Optional Challenge:_
1. Create a function that prints formatted outputs from the database tables
1. Create a single function that can be called to print for any table/database

In [ ]:
from shutil import copyfile

copyfile("rock_store_backup.db", "rock_store.db")
del copyfile

In [ ]:
from functools import reduce
import sqlite3
from contextlib import closing

def read_csv(filename: str) -> list[tuple]:
    with open(filename, encoding = "utf8") as equipment:
        return [tuple(line.rstrip("\n").split(",")) for line in equipment.readlines()]
    
def print_table(cursor: sqlite3.Cursor, table: str):
    rows: list[tuple] = cursor.execute(f"SELECT * FROM {table}").fetchall()
    
    number_columns = len(rows[0])
    column_lengths: list[int] = []
    for column in range(number_columns):
        lengths = [len(str(row[column])) for row in rows]
        column_lengths.append(max(lengths))

    for row in rows:
        print("|", end="")
        for column in range(number_columns):
            print(f" {row[column]:{column_lengths[column]}}", end= " |")
        print()
        

with closing(sqlite3.connect("rock_store.db")) as connection:
    with closing(connection.cursor()) as cursor:
        create_new_table = """
        CREATE TABLE equipment
        (
            id INTEGER,
            name TEXT,
            category TEXT,
            quantity INTEGER,
            price REAL
        )
        """
        cursor.execute(create_new_table)

        items = read_csv("new_equipment.csv")
        
        # Insert the items
        for item in items:
            cursor.execute("INSERT INTO equipment VALUES(?, ?, ?, ?, ?)", item)
        connection.commit()

        # Output the entries
        print("This is the content of the new equipment table")
        print_table(cursor, "equipment")

        # Update the prices
        updated_rocks = read_csv("updated_prices.csv")

        for rock in updated_rocks:
            update_command = """
                             UPDATE rocks SET price = ?
                             WHERE id = ?
                             """
            cursor.execute(update_command, (rock[3], rock[0]))
        connection.commit()

        # Display the new rock list
        print("\nThis is the content of the rocks table")
        print_table(cursor, "rocks")

        # Get your rocks off (the table)
        no_stock = cursor.execute("SELECT (id) FROM rocks WHERE stock = 0").fetchall()

        for rock in no_stock:
            cursor.execute("DELETE FROM rocks WHERE id = ?", rock)

        # Print rocks again
        print("\nUnstocked rocks were removed. Here is the updated rocks table:")
        print_table(cursor, "rocks")

In [ ]:
del connection
del create_new_table
del cursor
del item
del items
del no_stock
del rock
del update_command
del updated_rocks

## Lab 03.03
1. Modify `Student` and include two attributes
   * a private (`__`) attribute "name"
   * a protected (`_`) attribute "grade"
   * `grade` represents their current year or school
1. Practice accessing and mutating the variables with and without the getter/setter methods

_Optional Challenge:_
1. Add a `__str__` method that describes the state of a student object
1. Cause an error if the grade value is less than 1 or greater than 12
   * Be sure it works on new instances and when changing the grade of a current instance

In [ ]:
import sys

class Student:
    __name: str
    _grade: int

    def __init__(self, name: str, grade: int):
        self.__name = name
        self.grade = grade

    @property
    def name(self) -> str:
        return self.__name
    
    @property
    def grade(self) -> int:
        return self._grade
    
    @grade.setter
    def grade(self, value) -> None:
        if value >= 1 and value <= 12:
            self._grade = value
        else:
            raise ValueError("Grade cannot be below 1 or above 12")
    
    def attend_class(self) -> None:
        pass

student = Student("Nicholas", 5)

print("Since Student's name has two underscores, attempting to access it will get an attribute error")
print(f"If the access succeeds we will see 'The student's name is {student._Student__name}'")
print("Otherwise we will catch the exception.")
try:
    print(f"The student's name is {student.__name}")
except AttributeError:
    print("We got an AttributeError!")
print()

print("Now we will access Student's grade without a getter")
print(f"The student's grade is {student._grade}")
print()

print("Now let's try to set the properties!")
print("'name' does not have a property setter, so we will get an exception")
try:
    student.name = "New Name"
    print(f"The student's new name is {student.name}")
except AttributeError:
    print("We got an AttributeError")

print()
print("Now let's set 'grade'!")

old_grade = student.grade
student.grade = 8

print(f"Student's grade was changed from {old_grade} to {student.grade}")
print()

print("Now let's set 'grade' to something out of bounds")
try:
    student.grade = 100
except ValueError:
    print("We got a ValueError!")
    print(f"The exception says '{sys.exception()}'")

In [ ]:
del old_grade
del student

## Inheritance

### Overloading in Python
* Overloading doesn't work in Python
* Causes an error when the first method is called
* **Note:** Interpreter recognizes the **LAST ONE POSITIONALLY**

In [ ]:
import sys

class SpaceShip:
    def go(self, speed):
        print(f"Go with only speed {speed}")

    def go(self, speed, warp_faster):
        print(f"Go with speed and warp factor {speed}, {warp_faster}")

SpaceShip().go(1, True)
try:
    SpaceShip().go(1)
except:
    print(sys.exception())

## Lab 03.04 - Student Types
1. Create two subclasses of `Student`
   * `HighSchoolStudent`
   * `CollegeStudent`
1. Both classes need an `attend_class` method
   * For high school, print a message that attendance is required
   * For college, print a message that attendance is optional
1. Both classes should override `__str__`
1. The `grade` attribute can now be as high as 16
1. Create a list of at least 6 students in various grades, loop through it and call their methods

_Optional Challenge:_
1. College students can do research. Use multiple inheritance to solve this. Also, how can we handle it in our loop from step 5?
1. Populate the list of students programmatically

In [ ]:
from abc import ABCMeta, abstractmethod

class Researcher(metaclass = ABCMeta):
    @abstractmethod
    def do_reasearch(self) -> None:
        pass

In [ ]:
class HighSchoolStudent(Student):
    def attend_class(self):
        print("Attendance is mandatory")

    def __str__(self) -> str:
        return f"HighSchoolStudent(name: {self.name}, grade: {self.grade})"

class CollegeStudent(Student, Researcher):
    @property
    def grade(self) -> int:
        return self._grade

    @grade.setter
    def grade(self, value) -> None:
        if value >= 1 and value <= 16:
            self._grade = value
        else:
            raise ValueError("College grades can only be between 1 and 16")

    def attend_class(self):
        print("Attendance is optional")

    def do_reasearch(self) -> None:
        print("Searching for the meaning of life...")

    def __str__(self) -> str:
        return f"CollegeStudent(name: {self.name}, grade: {self.grade})"

In [ ]:
from faker import Faker
from random import choice, randint, seed

def get_random_students(number: int) -> list[Student]:
    fake = Faker()
    fake.seed_instance(0)
    seed(0)

    STUDENT_FACTORY: list[function] = [
        lambda: CollegeStudent(fake.first_name(), randint(13, 16)),
        lambda: HighSchoolStudent(fake.first_name(), randint(1, 12))
        ]

    return [choice(STUDENT_FACTORY)() for _ in range(number)]

for student in get_random_students(6):
    print(student)
    student.attend_class()
    
    if isinstance(student, Researcher):
        student.do_reasearch()
    print()

In [ ]:
del student
del choice
del randint
del seed

## Lab 03.05 - StudentAbc
1. Convert the Student class from the previous lab to an `Abstract Base Class`
1. What method(s) should be abstract?
1. How does this change affect the design and extensibility of your code?

_Optional Challenge:_
1. Create an abstract property `age` in the `StudentAbc` and demonstrate how to call it

In [ ]:
from abc import ABC, abstractmethod

class StudentAbc(ABC):
    __name: str
    __grade: int

    def __init__(self, name: str, grade: int) -> None:
        self.__name = name
        self.grade = grade

    @property
    def name(self) -> str:
        return self.__name
    
    @property
    def grade(self) -> int:
        return self.__grade
    
    @grade.setter
    def grade(self, value) -> None:
        self.__grade = value

    @property
    @abstractmethod
    def age(self) -> int:
        pass

    @age.setter
    @abstractmethod
    def age(self, value) -> None:
        pass

    @abstractmethod
    def attend_class(self):
        pass

In [ ]:
class CollegeStudent(StudentAbc):
    __age: int

    def __init__(self, name: str, grade: int, age: int) -> None:
        super().__init__(name, grade)
        self.age = age

    @property
    def age(self):
        return self.__age
    
    @age.setter
    def age(self, value):
        if value >= 18:
            self.__age = value
        else:
            raise ValueError("College Students must be at least 18")
    
    def attend_class(self):
        print("Attendance is optional")

college_student = CollegeStudent("Nicholas", grade=5, age=27)

try:
    print("The subclass does not allow ages below 18. We're going to try to set the age to 5")
    college_student.age = 5
except ValueError:
    print("We got a a value error when we tried that")

In [ ]:
del college_student

## Lab 04.01 - Docstring Practice
1. Create a simple function that takes two integers, multiplies them, and returns the result
1. Practice documenting the function with docstrings

In [ ]:
def multiply(a: int, b: int) -> int:
    """
    Accepts two integers and multiplies them together.

    Returns the result
    """
    return a * b

In [ ]:
import pydoc

with open("out.html", "w") as out:
    out.write(pydoc.html.document(multiply))

In [ ]:
import os

del out
os.remove("out.html")

## Lab 04.02 - Monkey Patching Exploration
1. Create a class called IRS
1. Create a method `calculate_tax(self, income)`
1. The functionallity is not ready yet, just leave the method empty
1. Create a `Client` class with a `calculate_my_income_tax(self, income)` method
1. This class uses a reference of IRS to call `calculate_tax`
1. Create a `mock_calculate_tax(self, income)` method that just returns a flat 20% tax on any income
1. Use monkey patching to ensure the right method is invoked

_Optional Challenge:_
1. Try monkey patching a module

In [ ]:
class IRS:
    def calculate_tax(self, income: float) -> float:
        pass

class Client:
    __irs: IRS

    def __init__(self) -> None:
        self.__irs = IRS()

    def calculate_my_income_tax(self, income: float):
        return self.__irs.calculate_tax(income)

def mock_calculate_tax(self, income: float) -> float:
    return income * .20

def monkey_patch() -> None:
    IRS.calculate_tax = mock_calculate_tax

client: Client = Client()
monkey_patch()
print(f"After monkey patching, the income tax amount is {client.calculate_my_income_tax(100)}")

In [ ]:
import random

print("Now we're going to always return the number 5 from 'randint'")
random.randint = lambda a, b: 5
print(f"Here is a list of 10 'random' integers {[random.randint(0, 100) for _ in range(10)]}")


In [ ]:
del client